In [ ]:
!git clone https://github.com/gerzin/IronySarcasmDetectorIT.git
!cd /content/IronySarcasmDetectorIT
!git pull
!cd ..

Cloning into 'IronySarcasmDetectorIT'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 305 (delta 161), reused 169 (delta 74), pack-reused 0
Receiving objects: 100% (305/305), 1.14 MiB | 8.83 MiB/s, done.
Resolving deltas: 100% (161/161), done.
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
import sys
sys.path.append('/content/IronySarcasmDetectorIT')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


# DEFINE MODELS

In [ ]:
%%capture
!pip install emoji
!pip install transformers

In [ ]:
from preprocessing.pipeline import ItalianTweetsPreprocessingPipeline
pre_pipeline = ItalianTweetsPreprocessingPipeline(to_lowercase=False)
df = pd.read_csv("/content/IronySarcasmDetectorIT/datasets/training_ironita2018.csv")
#pre-processing the data
df = pre_pipeline.apply(df)

In [ ]:
from pathlib import Path
class ModelsConfig:
    SEQUENCE_LENGTH = 50
    BERT_ITA_XXL_CASED = "dbmdz/bert-base-italian-xxl-cased"
    BERT_TOKENIZER_LENGTH = 80
    BERT_MODEL_NAME = "bertlstm.h5"

In [ ]:
from transformers import TFBertModel, AutoTokenizer
import numpy as np
import tensorflow as tf


def get_bert_tokenizer(model_url=ModelsConfig.BERT_ITA_XXL_CASED, tok_len=ModelsConfig.BERT_TOKENIZER_LENGTH):
    tokenizer = AutoTokenizer.from_pretrained(model_url, add_special_tokens=True, max_length=tok_len,
                                              pad_to_max_length=True)

    return tokenizer

def tokenize(sentences, tokenizer):
    input_ids, input_masks, input_segments = [], [], []
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=128, pad_to_max_length=True,
                                       return_attention_mask=True, return_token_type_ids=True, truncation=True)
        input_ids.append(inputs['input_ids'])
        input_masks.append(inputs['attention_mask'])
        input_segments.append(inputs['token_type_ids'])

    return np.asarray(input_ids, dtype='int32'), np.asarray(input_masks, dtype='int32'), np.asarray(input_segments,
                                                                                                    dtype='int32')


def get_bert_gru_classifier(hidden_layers, model_url=ModelsConfig.BERT_ITA_XXL_CASED):
    with tf.device(device_name):
      bert = TFBertModel.from_pretrained(model_url)

      input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
      input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32')

      embedding_layer = bert(input_ids_in, attention_mask=input_masks_in)[0]
      
      first = True
      for layer in hidden_layers:
        if first:
           X =  tf.keras.layers.GRU(layer[0], return_sequences=layer[1])(embedding_layer)
           first = False
        else: 
          X =  tf.keras.layers.GRU(layer[0], return_sequences=layer[1])(X)
        if layer[2] != 0.0:
           X = tf.keras.layers.Dropout(layer[2])(X)
      
      X = tf.keras.layers.Dense(1, activation = 'sigmoid')(X)

      model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs=X)

      for layer in model.layers[:3]:
          layer.trainable = False

      return model

In [ ]:
tokenizer = get_bert_tokenizer()

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

# GRID SEARCH

In [ ]:
import itertools
from sklearn.utils import shuffle

VALIDATION_SPLIT = 0.1
df_gridSearch = shuffle(df)
validation_size = int(len(df)*VALIDATION_SPLIT)
x_train_gs = df_gridSearch['text'][validation_size:]
y_train_gs = df_gridSearch['irony'][validation_size:]
x_vali_gs = df_gridSearch['text'][:validation_size]
y_vali_gs = df_gridSearch['irony'][:validation_size]

x_train_gs = tokenize(x_train_gs, tokenizer)[:-1]
x_vali_gs = tokenize(x_vali_gs, tokenizer)[:-1]

hidden_layers = [
                 [(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)],
                 [(128, True, 0.5), (32, False, 0.2)],
                 [(128, True, 0.5), (16, False, 0.2)],
                 [(128, True, 0.5), (32, False, 0.0)],
                 [(128, False, 0.0)],
]

loss = ['binary_crossentropy']
epochs = [20]

combinations = list(itertools.product(*[hidden_layers, loss, epochs]))


early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')

validation_performance = []
index = 0
for combination in combinations:
  print(index)
  model = get_bert_gru_classifier(hidden_layers = combination[0])
  model.compile(loss=combination[1], optimizer='adam', metrics =['acc'])

  model.fit(x = (x_train_gs[0], x_train_gs[1]), y = np.asarray(y_train_gs), epochs=combination[2], verbose = 0, callbacks=[early_stopping_cb])
  result = model.evaluate(x = (x_vali_gs[0], x_vali_gs[1]), y = y_vali_gs)
  performance = [combination, dict(zip(model.metrics_names, result))] 
  print(performance)
  validation_performance.append(performance)
  index +=1 

metric = 'acc'
values = []

for i in range(len(validation_performance)):
  values.append((validation_performance[i][0], validation_performance[i][1][metric]))

values.sort(key=lambda tup: tup[1], reverse = True)


print('Best Hyperparameters Irony')
for v in values:
  print(v[0], v[1])

In [ ]:
import itertools
from sklearn.utils import shuffle

VALIDATION_SPLIT = 0.1
df_gridSearch = shuffle(df)
validation_size = int(len(df)*VALIDATION_SPLIT)
x_train_gs = df_gridSearch['text'][validation_size:]
y_train_gs = df_gridSearch['sarcasm'][validation_size:]
x_vali_gs = df_gridSearch['text'][:validation_size]
y_vali_gs = df_gridSearch['sarcasm'][:validation_size]

x_train_gs = tokenize(x_train_gs, tokenizer)[:-1]
x_vali_gs = tokenize(x_vali_gs, tokenizer)[:-1]

hidden_layers = [
                 [(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)],
                 [(128, True, 0.5), (32, False, 0.2)],
                 [(128, True, 0.5), (16, False, 0.2)],
                 [(128, True, 0.5), (32, False, 0.0)],
                 [(128, False, 0.0)],
]

loss = ['binary_crossentropy']
epochs = [20]

combinations = list(itertools.product(*[hidden_layers, loss, epochs]))


early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')

validation_performance = []
index = 0
for combination in combinations:
  print(index)
  model = get_bert_gru_classifier(hidden_layers = combination[0])
  model.compile(loss=combination[1], optimizer='adam', metrics =['acc'])

  model.fit(x = (x_train_gs[0], x_train_gs[1]), y = np.asarray(y_train_gs), epochs=combination[2], verbose = 0, callbacks=[early_stopping_cb])
  result = model.evaluate(x = (x_vali_gs[0], x_vali_gs[1]), y = y_vali_gs)
  performance = [combination, dict(zip(model.metrics_names, result))] 
  print(performance)
  validation_performance.append(performance)
  index +=1 

metric = 'acc'
values = []

for i in range(len(validation_performance)):
  values.append((validation_performance[i][0], validation_performance[i][1][metric]))

values.sort(key=lambda tup: tup[1], reverse = True)


print('Best Hyperparameters Sarcasm')
for v in values:
  print(v[0], v[1])

In [ ]:
#RISULTATI GREED SEARCH

irony_evaluations = [
[([(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)], 'binary_crossentropy', 10), {'loss': 0.5113739371299744, 'acc': 0.7959697842597961}],
[([(128, True, 0.5), (32, False, 0.2)], 'binary_crossentropy', 10), {'loss': 0.7333301901817322, 'acc': 0.7304785847663879}],
[([(128, True, 0.5), (16, False, 0.2)], 'binary_crossentropy', 10), {'loss': 0.6317560076713562, 'acc': 0.75314861536026}],
[([(128, True, 0.5), (32, False, 0.0)], 'binary_crossentropy', 10), {'loss': 0.6505503058433533, 'acc': 0.750629723072052}],
[([(128, False, 0.0)], 'binary_crossentropy', 10), {'loss': 0.7161769270896912, 'acc': 0.7178841233253479}],
[([(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)], 'binary_crossentropy', 20), {'loss': 1.0052123069763184, 'acc': 0.7279596924781799}],
[([(128, True, 0.5), (32, False, 0.2)], 'binary_crossentropy', 20), {'loss': 1.2487032413482666, 'acc': 0.7052896618843079}],
[([(128, True, 0.5), (16, False, 0.2)], 'binary_crossentropy', 20), {'loss': 1.0360223054885864, 'acc': 0.7304785847663879}],
[([(128, True, 0.5), (32, False, 0.0)], 'binary_crossentropy', 20), {'loss': 1.0536763668060303, 'acc': 0.732997477054596}],
[([(128, False, 0.0)], 'binary_crossentropy', 20), {'loss': 1.0814464092254639, 'acc': 0.7204030156135559}]
]

sarcasm_evaluations = [
[([(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)], 'binary_crossentropy', 10), {'loss': 0.6838492155075073, 'acc': 0.7279596924781799}],
[([(128, True, 0.5), (32, False, 0.2)], 'binary_crossentropy', 10), {'loss': 0.7170308828353882, 'acc': 0.748110830783844}],
[([(128, True, 0.5), (16, False, 0.2)], 'binary_crossentropy', 10), {'loss': 0.6712967157363892, 'acc': 0.743073046207428}],
[([(128, True, 0.5), (32, False, 0.0)], 'binary_crossentropy', 10), {'loss': 0.7501422762870789, 'acc': 0.7078085541725159}],
[([(128, False, 0.0)], 'binary_crossentropy', 10), {'loss': 0.8842348456382751, 'acc': 0.6851385235786438}],
[([(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)], 'binary_crossentropy', 20), {'loss': 1.1755075454711914, 'acc': 0.6801007390022278}],
[([(128, True, 0.5), (32, False, 0.2)], 'binary_crossentropy', 20), {'loss': 1.3208746910095215, 'acc': 0.7002518773078918}],
[([(128, True, 0.5), (16, False, 0.2)], 'binary_crossentropy', 20), {'loss': 1.2237690687179565, 'acc': 0.7027707695960999}]
]

In [ ]:
metric = 'acc'
values = []

for i in range(len(sarcasm_evaluations)):
  values.append((sarcasm_evaluations[i][0], sarcasm_evaluations[i][1][metric]))

values.sort(key=lambda tup: tup[1], reverse = True)


print('Best Hyperparameters Sarcasm')
for v in values:
  print(v[0], v[1])

In [ ]:
metric = 'acc'
values = []

for i in range(len(irony_evaluations)):
  values.append((irony_evaluations[i][0], irony_evaluations[i][1][metric]))

values.sort(key=lambda tup: tup[1], reverse = True)


print('Best Hyperparameters Irony')
for v in values:
  print(v[0], v[1])

Best Hyperparameters Irony
([(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)], 'binary_crossentropy', 10) 0.7959697842597961
([(128, True, 0.5), (16, False, 0.2)], 'binary_crossentropy', 10) 0.75314861536026
([(128, True, 0.5), (32, False, 0.0)], 'binary_crossentropy', 10) 0.750629723072052
([(128, True, 0.5), (32, False, 0.0)], 'binary_crossentropy', 20) 0.732997477054596
([(128, True, 0.5), (32, False, 0.2)], 'binary_crossentropy', 10) 0.7304785847663879
([(128, True, 0.5), (16, False, 0.2)], 'binary_crossentropy', 20) 0.7304785847663879
([(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)], 'binary_crossentropy', 20) 0.7279596924781799
([(128, False, 0.0)], 'binary_crossentropy', 20) 0.7204030156135559
([(128, False, 0.0)], 'binary_crossentropy', 10) 0.7178841233253479
([(128, True, 0.5), (32, False, 0.2)], 'binary_crossentropy', 20) 0.7052896618843079


# FITTING THE BEST MODEL

In [ ]:
from custom_metrics import computePerformanceTaskB_2output
df_train = pd.read_csv("/content/IronySarcasmDetectorIT/datasets/training_ironita2018.csv")
#pre-processing the data
df_train = pre_pipeline.apply(df_train)

df_test = pd.read_csv("/content/IronySarcasmDetectorIT/datasets/test_gold_ironita2018.csv")
df_test = pre_pipeline.apply(df_test)

x_train = df_train['text']
y_train = df_train[['irony','sarcasm']]

x_test = df_test['text']
y_test = df_test[['irony','sarcasm']]
x_test = tokenize(x_test, tokenizer)[:-1]
x_train = tokenize(x_train, tokenizer)[:-1]








/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Seed value
seed_value= 450

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

In [ ]:
best_combination_irony = ([(128, True, 0.5), (64, True, 0.2), (16, False, 0.0)], 'binary_crossentropy', 10)
model_irony = get_bert_gru_classifier(hidden_layers = best_combination_irony[0])
model_irony.compile(loss=best_combination_irony[1], optimizer='adam', metrics =['acc'])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')

model_irony.fit(x = (x_train[0], x_train[1]), y = np.asarray(y_train['irony']), epochs=best_combination_irony[2], verbose = 1, callbacks=[early_stopping_cb])

Downloading:   0%|          | 0.00/520M [00:00<?, ?B/s]

Some layers from the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-italian-xxl-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/10
125/125 [==============================] - 95s 590ms/step - loss: 0.6140 - acc: 0.6585
Epoch 2/10
125/125 [==============================] - 75s 601ms/step - loss: 0.5434 - acc: 0.7242
Epoch 3/10
125/125 [==============================] - 75s 601ms/step - loss: 0.4988 - acc: 0.7538
Epoch 4/10
125/125 [==============================] - 74s 593ms/step - loss: 0.4725 - acc: 0.7762
Epoch 5/10
125/125 [==============================] - 74s 591ms/step - loss: 0.4313 - acc: 0.7973
Epoch 6/10
125/125 [==============================] - 74s 592ms/step - loss: 0.3923 - acc: 0.8220
Epoch 7/10
125/125 [==============================] - 74s 592ms/step - loss: 0.3684 - acc: 0.8423
Epoch 8/10
125/125 [==============================] - 74s 590ms/step - loss: 0.3162 - acc: 0.8594
Epoch 9/10
125/125 [==============================] - 74s 590ms/step - loss: 0.2684 - acc: 0.8853
Epoch 10/10
125/125 [==============================] - 74s 589ms/step - loss: 0.2430 - acc: 0.8969


In [ ]:
best_combination_sarcasm = ([(128, True, 0.5), (32, False, 0.2)], 'binary_crossentropy', 10)
model_sarcasm = get_bert_gru_classifier(hidden_layers = best_combination_sarcasm[0])
model_sarcasm.compile(loss=best_combination_sarcasm[1], optimizer='adam', metrics =['acc'])
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')

model_sarcasm.fit(x = (x_train[0], x_train[1]), y = np.asarray(y_train['sarcasm']), epochs=best_combination_sarcasm[2], verbose = 1, callbacks=[early_stopping_cb])

Some layers from the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-italian-xxl-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/10
125/125 [==============================] - 88s 569ms/step - loss: 0.5350 - acc: 0.7576
Epoch 2/10
125/125 [==============================] - 72s 572ms/step - loss: 0.5032 - acc: 0.7667
Epoch 3/10
125/125 [==============================] - 71s 571ms/step - loss: 0.4804 - acc: 0.7740
Epoch 4/10
125/125 [==============================] - 72s 574ms/step - loss: 0.4549 - acc: 0.7888
Epoch 5/10
125/125 [==============================] - 72s 573ms/step - loss: 0.4480 - acc: 0.7956
Epoch 6/10
125/125 [==============================] - 71s 572ms/step - loss: 0.4282 - acc: 0.7956
Epoch 7/10
125/125 [==============================] - 71s 572ms/step - loss: 0.4004 - acc: 0.8177
Epoch 8/10
125/125 [==============================] - 71s 572ms/step - loss: 0.3724 - acc: 0.8298
Epoch 9/10
125/125 [==============================] - 72s 574ms/step - loss: 0.3330 - acc: 0.8476
Epoch 10/10
125/125 [==============================] - 72s 572ms/step - loss: 0.3027 - acc: 0.8682


In [ ]:
from custom_metrics import model_test, computePerformanceTaskB_2model
print("F1 Average Task A")
print(model_test(model_irony, x_test, y_test['irony']))
print("F1 Average Task B")
print(computePerformanceTaskB_2model(model_irony, model_sarcasm, x_test, y_test))

F1 Average Task A
0.7052438304202404
F1 Average Task B
0.49626910207369246
